# Urban Traffic Las Vegas Superbowl LVIII
Simulate the urban traffic from Las Vegas using [ArcGIS API for Python](https://developers.arcgis.com/python/).

In [ ]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

## Required Python modules
You need to install ArcGIS API for Python, please follow the [guide](https://developers.arcgis.com/python/guide/anaconda/).

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from datetime import date, datetime, timedelta
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from geourban.services import aggregate, simulations, top
from geourban.types import GridType, VehicleType

## Connect to ArcGIS Online

In [ ]:
gis = GIS()

## Create a map showing the city of Las Vegas

In [ ]:
vegas_map = gis.map('Las Vegas, Nevada', zoomlevel=13)
vegas_map

## Connect to geourban services

The host parameter must target the specific host like \"geoprotests.p.rapidapi.com\". Furthermore, the factory directly access `os.environ['x_rapidapi_key']` and uses the specified API key as a header parameter. Otherwise, `georapid.factory.EnvironmentClientFactory.create_client_with_host()` will raise a `ValueError`.

Please, check out the [RapidAPI Account Creation and Management Guide](https://docs.rapidapi.com/docs/account-creation-and-settings).

In [ ]:
client = EnvironmentClientFactory.create_client_with_host('geourban.p.rapidapi.com')

## Request and map the accumulated pedestrian traffic grid cells before and after the Superbowl LVIII
We request these hotspots for the city of Las Vegas by using the urban region code `USQ27768421`, the simulation datetimes e.g. `2024-02-11T12:00:00`, the vehicle type `Pedestrian`, and the grid type `agent`. The returned GeoJSON features represents the grid cells showing the pedestrian throughput.

In [ ]:
def map_pedestrian_traffic(client, simulation_datetime):
    vegas_region_code = 'USQ27768421'
    vehicle_type = VehicleType.PEDESTRIAN
    grid_type = GridType.AGENT
    pedestrian_cells = aggregate(client, vegas_region_code, simulation_datetime, vehicle_type, grid_type)
    pedestrian_fset = FeatureSet.from_geojson(pedestrian_cells)
    pedestrian_sdf = pedestrian_fset.sdf
    vegas_map = gis.map('Las Vegas, Nevada', zoomlevel=13)
    pedestrian_sdf.spatial.plot(vegas_map, renderer_type='c', method='esriClassifyNaturalBreaks', class_count=5, col='agent_count', cmap='YlOrRd')
    vegas_map.zoom_to_layer(pedestrian_sdf)
    return vegas_map

## Map of pedestrian traffic between 12:00 PM and 01:00 PM

In [ ]:
simulation_datetime = datetime.fromisoformat('2024-02-11T12:00:00')
map_pedestrian_traffic(client, simulation_datetime)

## Map of pedestrian traffic between 01:00 PM and 02:00 PM

In [ ]:
simulation_datetime = datetime.fromisoformat('2024-02-11T13:00:00')
map_pedestrian_traffic(client, simulation_datetime)

## Map of pedestrian traffic between 02:00 PM and 03:00 PM

In [ ]:
simulation_datetime = datetime.fromisoformat('2024-02-11T14:00:00')
map_pedestrian_traffic(client, simulation_datetime)

## Map of pedestrian traffic after the game between 10:00 PM and 11:00 PM

In [ ]:
simulation_datetime = datetime.fromisoformat('2024-02-11T22:00:00')
map_pedestrian_traffic(client, simulation_datetime)